In [0]:
rdd=sc.textFile("/FileStore/tables/weather_data.txt")

In [0]:
rdd.take(5)

Out[177]: ['23907 20150101  2.423  -98.08   30.62     2.2    -0.6     0.8     0.9     6.2     1.47 C     3.7     1.1     2.5    99.9    85.4    97.2   0.369   0.308 -99.000 -99.000 -99.000     7.0     8.1 -9999.0 -9999.0 -9999.0',
 '23907 20150102  2.423  -98.08   30.62     3.5     1.3     2.4     2.2     9.0     1.43 C     4.9     2.3     3.1   100.0    98.8    99.8   0.391   0.327 -99.000 -99.000 -99.000     7.1     7.9 -9999.0 -9999.0 -9999.0',
 '23907 20150103  2.423  -98.08   30.62    15.9     2.3     9.1     7.5     2.9    11.00 C    16.4     2.9     7.3   100.0    34.8    73.7   0.450   0.397 -99.000 -99.000 -99.000     7.6     7.9 -9999.0 -9999.0 -9999.0',
 '23907 20150104  2.423  -98.08   30.62     9.2    -1.3     3.9     4.2     0.0    13.24 C    12.4    -0.5     4.9    82.0    40.6    61.7   0.414   0.352 -99.000 -99.000 -99.000     7.3     7.9 -9999.0 -9999.0 -9999.0',
 '23907 20150105  2.423  -98.08   30.62    10.9    -3.7     3.6     2.6     0.0    13.37 C    14.7    -3.0

In [0]:
def transform(values):
        values=values.replace(" C","")
        values=values.split()
        values[1]=values[1][0:4]+"-"+values[1][4:6]+"-"+values[1][6:]
        return values

In [0]:
rdd=rdd.map(lambda values:transform(values))

In [0]:
rdd.take(1)

Out[180]: [['23907',
  '2015-01-01',
  '2.423',
  '-98.08',
  '30.62',
  '2.2',
  '-0.6',
  '0.8',
  '0.9',
  '6.2',
  '1.47',
  '3.7',
  '1.1',
  '2.5',
  '99.9',
  '85.4',
  '97.2',
  '0.369',
  '0.308',
  '-99.000',
  '-99.000',
  '-99.000',
  '7.0',
  '8.1',
  '-9999.0',
  '-9999.0',
  '-9999.0']]

In [0]:
schema = ["WBANNO","LST_DATE","CRX_VN","LONGITUDE","LATITUDE","T_DAILY_MAX","T_DAILY_MIN","T_DAILY_MEAN","T_DAILY_AVG"                    "P_DAILY_CALC","SOLARAD_DAILY","SUR_TEMP_DAILY_TYPE","SUR_TEMP_DAILY_MAX","SUR_TEMP_DAILY_MIN","SUR_TEMP_DAILY_AVG","RH_DAILY_MAX","RH_DAILY_MIN ","RH_DAILY_AVG"          ,"SOIL_MOISTURE_5_DAILY","SOIL_MOISTURE_10_DAILY","SOIL_MOISTURE_20_DAILY","SOIL_MOISTURE_50_DAILY","SOIL_MOISTURE_100_DAILY","SOIL_TEMP_5_DAILY","SOIL_TEMP_10_DAILY ","SOIL_TEMP_20_DAILY","SOIL_TEMP_50_DAILY","SOIL_TEMP_100_DAILY"]

In [0]:
df=rdd.toDF(schema)

In [0]:
df.show(5)

+------+----------+------+---------+--------+-----------+-----------+------------+-----------------------+-------------+-------------------+------------------+------------------+------------------+------------+-------------+------------+---------------------+----------------------+----------------------+----------------------+-----------------------+-----------------+-------------------+------------------+------------------+-------------------+
|WBANNO|  LST_DATE|CRX_VN|LONGITUDE|LATITUDE|T_DAILY_MAX|T_DAILY_MIN|T_DAILY_MEAN|T_DAILY_AVGP_DAILY_CALC|SOLARAD_DAILY|SUR_TEMP_DAILY_TYPE|SUR_TEMP_DAILY_MAX|SUR_TEMP_DAILY_MIN|SUR_TEMP_DAILY_AVG|RH_DAILY_MAX|RH_DAILY_MIN |RH_DAILY_AVG|SOIL_MOISTURE_5_DAILY|SOIL_MOISTURE_10_DAILY|SOIL_MOISTURE_20_DAILY|SOIL_MOISTURE_50_DAILY|SOIL_MOISTURE_100_DAILY|SOIL_TEMP_5_DAILY|SOIL_TEMP_10_DAILY |SOIL_TEMP_20_DAILY|SOIL_TEMP_50_DAILY|SOIL_TEMP_100_DAILY|
+------+----------+------+---------+--------+-----------+-----------+------------+--------------------

In [0]:
from pyspark.sql.types import DateType,FloatType
from pyspark.sql.functions import *

In [0]:
df1=df.select(col('LST_DATE').cast('date'),col('T_DAILY_MAX').cast('float'),col('T_DAILY_MIN').cast('float'))
df1.show(5)

+----------+-----------+-----------+
|  LST_DATE|T_DAILY_MAX|T_DAILY_MIN|
+----------+-----------+-----------+
|2015-01-01|        2.2|       -0.6|
|2015-01-02|        3.5|        1.3|
|2015-01-03|       15.9|        2.3|
|2015-01-04|        9.2|       -1.3|
|2015-01-05|       10.9|       -3.7|
+----------+-----------+-----------+
only showing top 5 rows



In [0]:
df1.agg( \
    max('T_DAILY_MAX').alias('max_temp'),\
    min('T_DAILY_MIN').alias('min_temp')\
).show()

+--------+--------+
|max_temp|min_temp|
+--------+--------+
|    36.0|    -7.9|
+--------+--------+



In [0]:
df1.groupBy(month("LST_DATE").alias('month')).agg( \
    max('T_DAILY_MAX').alias('max_temp'),\
    min('T_DAILY_MIN').alias('min_temp')\
).show()

+-----+--------+--------+
|month|max_temp|min_temp|
+-----+--------+--------+
|    1|    26.5|    -7.9|
|    3|    29.1|    -3.2|
|    4|    30.8|     8.0|
|    2|    26.6|    -3.5|
|    6|    33.6|     0.0|
|    5|    31.1|    14.3|
|    7|    36.0|    19.8|
+-----+--------+--------+

